# Import Data and Packages

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics
import tensorflow as tf
import tensorflow.keras.layers as layers
import os
import time
pd.options.display.max_columns = 1200
pd.options.display.max_rows = 1200
tf.enable_eager_execution()

In [78]:
df = pd.read_csv('data/autoperf_filtered_1000000ch', header=0).sample(frac=1)

In [79]:
df = df[df['zero_execName'] == 'nek5000'].copy()

# Split Train/Test and Select Features

In [80]:
# features = ['zero_stdMpiIoTime']
features = df.columns.tolist()
remove = ['zero_execName', 'run_date', 'zero_userName', 'zero_threadMode', 'min_userName', 'min_execName', 'min_threadMode', 'max_execName', 'max_userName', 'max_threadMode', 'av_execName', 'av_userName', 'av_threadMode']
for re in remove:
    features.remove(re)
df = df[features]

In [81]:
train_size = .8
train = df.head(int(df.size * train_size))
test = df.tail(int(df.size * (1-train_size)))

# Some global variables we'll need
EPOCHS = 5
BATCH_SIZE = 512
dim = df.shape[1]
num_examples_to_generate = 256

seed = tf.random.normal([num_examples_to_generate, noise_dim])
train_dataset = tf.data.Dataset.from_tensor_slices(tf.cast(train.values, tf.float32)).batch(BATCH_SIZE)

# Make Generator and Discriminator

In [82]:
def build_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(dim, input_shape=(dim,)))
    model.add(layers.BatchNormalization(momentum=.8))
    model.add(layers.LeakyReLU(alpha=.2))

    model.add(layers.Dense(2 * dim))
    model.add(layers.BatchNormalization(momentum=.8))
    model.add(layers.LeakyReLU(alpha=.2))

    model.add(layers.Dense(4 * dim))
    model.add(layers.BatchNormalization(momentum=.8))
    model.add(layers.LeakyReLU(alpha=.2))
    
    model.add(layers.Dense(dim))

    return model

In [83]:
def build_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(dim, input_shape=(dim,)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(2 * dim))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(dim, activation='sigmoid'))
    
    return model

In [84]:
tf.reset_default_graph()
generator = build_generator_model()
discriminator = build_discriminator_model()

# Define Losses, Optimizers, and Checkpoints

In [85]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(.00001)
discriminator_optimizer = tf.keras.optimizers.Adam(.000001)

In [86]:
# checkpoint_dir = './training_checkpoints'
# checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
# checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
#                                  discriminator_optimizer=discriminator_optimizer,
#                                  generator=generator,
#                                  discriminator=discriminator)

# Training Time!

In [87]:
def train_step(samples):
    noise = tf.random.normal([BATCH_SIZE, dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_records = generator(noise, training=True)

      real_output = discriminator(samples, training=True)
      fake_output = discriminator(generated_records, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss

In [88]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()
    
    for batch in dataset:
      gen_loss, disc_loss = train_step(batch)

    print("Generator Loss: ", gen_loss)
    print("Discriminator Loss: ", disc_loss)
    
#     # Save the model every 15 epochs
#     if (epoch + 1) % 15 == 0:
#       checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [89]:
%%time
train(train_dataset, EPOCHS)

Generator Loss:  tf.Tensor(0.46864343, shape=(), dtype=float32)
Discriminator Loss:  tf.Tensor(1.5161161, shape=(), dtype=float32)
Time for epoch 1 is 364.94622468948364 sec
Generator Loss:  tf.Tensor(0.4735946, shape=(), dtype=float32)
Discriminator Loss:  tf.Tensor(1.5115825, shape=(), dtype=float32)
Time for epoch 2 is 365.159321308136 sec
Generator Loss:  tf.Tensor(0.4733104, shape=(), dtype=float32)
Discriminator Loss:  tf.Tensor(1.5203242, shape=(), dtype=float32)
Time for epoch 3 is 367.79504776000977 sec
Generator Loss:  tf.Tensor(0.4773119, shape=(), dtype=float32)
Discriminator Loss:  tf.Tensor(1.5176995, shape=(), dtype=float32)
Time for epoch 4 is 365.05380964279175 sec
Generator Loss:  tf.Tensor(0.48033404, shape=(), dtype=float32)
Discriminator Loss:  tf.Tensor(1.5131345, shape=(), dtype=float32)
Time for epoch 5 is 365.0112781524658 sec
CPU times: user 30min 27s, sys: 47.4 ms, total: 30min 27s
Wall time: 30min 27s


In [90]:
res = generator(tf.random.normal([5, dim]))
print(res)

tf.Tensor(
[[ 0.9252443  -4.2817936  -4.008192   ...  0.6407013   0.62792885
  -3.4423625 ]
 [ 2.0405703  -3.3222964  -2.3516433  ... -0.2812785   0.15884714
  -4.08027   ]
 [ 2.0428686  -5.302197   -2.4947195  ...  0.51397693  0.4424576
  -3.7906888 ]
 [ 2.8683991  -4.1444473  -3.6069608  ...  0.34208918  0.2937318
  -4.345035  ]
 [ 2.615412   -3.639512   -4.5312057  ...  0.11355054 -0.57811034
  -4.23321   ]], shape=(5, 1044), dtype=float32)
